In [3]:
import numpy as np
import cv2
import mediapipe as mp
import sys
import os
import pandas as pd

import multimodel
from multimodel import MultiModelDetector

### small_version

In [4]:
behaviors = multimodel.behaviors
str2int = multimodel.str2int
behavior_idx = multimodel.behavior_idx
get_label = multimodel.get_label

for behavior in behaviors:
    mmd = MultiModelDetector(behavior)
    
    use_dir = multimodel.Behavior_Information(behavior).behaviors
    label_dict = get_label[behavior_idx[behavior]]

    idx1, idx2 = mmd.get_idx()
    vector_idx1, vector_idx2 = mmd.get_vector_idx()

    actions_datas = []
    actions_names = [] 
    labels = []

    datasets_directory = 'datasets_locations_csvs'
    for action_dir in os.listdir(datasets_directory):
        if action_dir not in use_dir:
            continue

        action_path = os.path.join(datasets_directory, action_dir)

        for filename in os.listdir(action_path):
            if not filename.endswith(".csv"):  # CSV 파일인 경우에만 처리
                continue

            data = pd.read_csv(os.path.join(action_path, filename))
            for row in data.itertuples(index=False):
                row_data = np.zeros((91, 3))
                for j in range(91):
                    # 문자열을 배열로 변환하여 저장
                    try:
                        # 문자열에서 불필요한 문자 제거 후 배열로 변환
                        row_data[j] = np.array([float(x) for x in row[j][1:-1].split()] if row[j] else np.array([0.0, 0.0, 0.0]))
                    except (ValueError, TypeError):
                        row_data[j] = np.array([0.0, 0.0, 0.0])  # 변환할 수 없는 경우 0으로 처리
                        
                if 'right' in action_dir and any(np.array_equal(row_data[i], np.array([0.0, 0.0, 0.0])) for i in range(15)):
                    continue
                if 'left' in action_dir and any(np.array_equal(row_data[i], np.array([0.0, 0.0, 0.0])) for i in range(15, 30)):
                    continue
                if 'both' in action_dir and any(np.array_equal(row_data[i], np.array([0.0, 0.0, 0.0])) for i in range(30)):
                    continue

                cheek_meets = mmd.get_meet_finger_cheek(row_data)
                if action_dir == 'cheek_right' and not np.array_equal(cheek_meets, np.array([1, 0])):
                    continue
                if action_dir == 'cheek_left' and not np.array_equal(cheek_meets, np.array([0, 1])):
                    continue

                angles = mmd.get_angle_numpy(row_data)
                distances = mmd.get_distances_numpy(row_data)
                row_train_data = np.concatenate((angles, distances, cheek_meets))

                actions_datas.append(row_train_data)
                action_dir = action_dir
                actions_names.append(action_dir)
                labels.append(label_dict[action_dir])


    rha = [f'RightHandAngle{i+1}' for i in range(15)]
    lha = [f'LeftHandAngle{i+1}' for i in range(15)]
    poa = [f'PoseAngle{i+1}' for i in range(16)]
    faa = [f'FaceAngle{i+1}' for i in range(16)]
    pod = [f'PoseDistance{i+1}' for i in range(12)]
    chb = ['RightCheek', 'LeftCheek']
    

    column_names = rha + lha + poa + faa + pod + chb
    df = pd.DataFrame(actions_datas, columns=column_names)  # 각도 데이터를 DataFrame으로 변환하고 열 이름을 지정
    df['name'] = actions_names  # 'name' 열에 디렉토리명 추가
    df['label'] = labels

    csv_directory = 'datasets_processing_data_csvs'
    if not os.path.exists(csv_directory):
        os.makedirs(csv_directory)
    filename = os.path.join(csv_directory, f'{behavior}_small_output.csv')
    df.to_csv(filename, index=False)  # CSV 파일로 저장


C:\Users\user\OneDrive\바탕 화면\캡스톤\Last_Last_capstone_code\multimodel.py:187: RuntimeWarning: invalid value encountered in divide
  v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]  # Normailize v (v의 길이를 1로 만듦)
C:\Users\user\OneDrive\바탕 화면\캡스톤\Last_Last_capstone_code\multimodel.py:187: RuntimeWarning: invalid value encountered in divide
  v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]  # Normailize v (v의 길이를 1로 만듦)
C:\Users\user\OneDrive\바탕 화면\캡스톤\Last_Last_capstone_code\multimodel.py:187: RuntimeWarning: invalid value encountered in divide
  v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]  # Normailize v (v의 길이를 1로 만듦)
C:\Users\user\OneDrive\바탕 화면\캡스톤\Last_Last_capstone_code\multimodel.py:187: RuntimeWarning: invalid value encountered in divide
  v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]  # Normailize v (v의 길이를 1로 만듦)
C:\Users\user\OneDrive\바탕 화면\캡스톤\Last_Last_capstone_code\multimodel.py:187: RuntimeWarning: invalid value encountered in divide
  v = v / np.linalg.norm(v, axis